In [ ]:
# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab


In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.6)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
assert torch.__version__.startswith("1.6")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

import requests
from firebase_admin import credentials, initialize_app, storage

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%%time
!rm -r frames/*
!mkdir frames/

In [ ]:
!pwd

In [ ]:
video = "/content/WhatsApp Video 2020-10-03 at 5.03.58 PM.mp4"

cap = cv2.VideoCapture(video)
cnt = 0

if(cap.isOpened() == False):
  print("Error opening video stream or file")

ret,first_frame = cap.read()

while(cap.isOpened()):
  ret,frame = cap.read()
  if(ret == True):
    cv2.imwrite('/content/drive/My Drive/social_distancing/Dataset/'+str(cnt)+'.png',frame)
    cnt = cnt + 1
    if(cnt == 750):
      break
  else:
    break

In [ ]:
FPS = cap.get(cv2.CAP_PROP_FPS)
print(FPS)

In [ ]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_C4_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_C4_3x.yaml")
predictor = DefaultPredictor(cfg)

In [ ]:
def mid_point(img,person,idx):
  x1,y1,x2,y2 = person[idx]
  _=cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)

  x_mid = int((x1+x2)/2)
  y_mid = int(y2)
  mid = (x_mid, y_mid)

  _=cv2.circle(img,mid,5,(0,255,0),-1)
  cv2.putText(img, str(idx), mid, cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

  return mid

In [ ]:
%%time
from scipy.spatial import distance
def compute_distance(midpoints,num):
  dist = np.zeros((num,num))
  for i in range(num):
    for j in range(i+1, num):
      if i!=j:
        dst = distance.euclidean(midpoints[i], midpoints[j])
        dist[i][j] = dst
  return dist

In [ ]:
def find_closest(dist,num,thresh):
  p1 = []
  p2 = []
  d = []
  for i in range(num):
    for j in range(i,num):
      if((i!=j) & (dist[i][j] <= thresh)):
        p1.append(i)
        p2.append(j)
        d.append(dist[i][j])
  return p1,p2,d

In [ ]:
def change_2_red(img,person,p1,p2,img_name,name,cnt):
  risky = np.unique(p1+p2)
  flag = 0
  if(len(risky)>0):
    flag = 1
  for i in risky:
    x1,y1,x2,y2 = person[i]
    _ = cv2.rectangle(img,(x1,y1),(x2,y2),(0,0,255),2)

  #cv2.imwrite("")
  cv2.imwrite('/content/frames/'+name,img)
  #cv2.imread('/content/frames/'+name,img)

  img_name = '/content/frames/'+name

  #gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
  # histogram = cv2.calcHist([img], [0],  
  #                        None, [256], [0, 256]) 
  
  # #gray_image1 = cv2.cvtColor(last_img, cv2.COLOR_BGR2GRAY) 
  # histogram1 = cv2.calcHist([last_img], [0],  
  #                         None, [256], [0, 256]) 
  
  

  # c1 = 0

  # i = 0
  # while i<len(histogram) and i<len(histogram1): 
  #     c1+=(histogram[i]-histogram1[i])**2
  #     i+= 1
  # c1 = c1**(1 / 2)
  # print(c1) 
  #print('last_immg = /content/drive/My Drive/social_distancing/Dataset/'+str(i-1)+'.png')
  #print('img = /content/frames/'+name)
  print("i =",i)
  if(flag == 1 and cnt%30 == 0 ):
    post_request(img_name,"A")
  return img

In [ ]:
import os
import re

names=os.listdir('/content/drive/My Drive/social_distancing/Dataset/')
names.sort(key=lambda f: int(re.sub('\D', '', f)))

In [ ]:
def initialize():
  from firebase_admin import credentials, initialize_app, storage
  # Init firebase with your credentials
  cred = credentials.Certificate("/content/c19watcher-bd93edd80c0e.json")
  initialize_app(cred, {'storageBucket': 'c19watcher.appspot.com'})


In [ ]:
initialize()

In [ ]:
def post_request(f_name,loc):
  fileName = f_name
  bucket = storage.bucket()
  blob = bucket.blob(fileName)
  blob.upload_from_filename(fileName)

  # Opt : if you want to make public access from the URL
  blob.make_public()

  print("your file url", blob.public_url)

  url = "https://c19watcher.firebaseio.com/SocialDistancing.json"
  no = loc
  payload = '{"Location": '+ no +'}'
  headers = {
    'Content-Type': 'application/json'
  }

  response = requests.post(url, json={"Location": no, "ImageUrl": blob.public_url, "Mask" : 0,"SD" : 1,"Temp" : 0})

  print(response.text.encode('utf8'))

In [ ]:
def find_closest_people(name,thresh,cnt):

  img = cv2.imread('/content/drive/My Drive/social_distancing/Dataset/'+name)
  img_name = '/content/drive/My Drive/social_distancing/Dataset/'+name
  outputs = predictor(img)
  classes=outputs['instances'].pred_classes.cpu().numpy()
  bbox=outputs['instances'].pred_boxes.tensor.cpu().numpy()
  ind = np.where(classes==0)[0]
  person=bbox[ind]
  midpoints = [mid_point(img,person,i) for i in range(len(person))]
  num = len(midpoints)
  dist= compute_distance(midpoints,num)
  p1,p2,d=find_closest(dist,num,thresh)
  # if(i == 0):
  #   last_img = cv2.imread("/content/drive/My Drive/social_distancing/Dataset/0.png")
  # else:
  #   last_img = cv2.imread('/content/frames/'+str(i-1)+'.png')
  img = change_2_red(img,person,p1,p2,img_name,name,cnt)
  #print(flag)
  return 0

In [ ]:
from tqdm import tqdm
thresh=100
last_img = cv2.imread("/content/drive/My Drive/social_distancing/Dataset/0.png")
cnt = 0
for i in tqdm(range(len(names))):
  try:
    
    _= find_closest_people(names[i],thresh,cnt)
    cnt = cnt+1
    #last_img = cv2.imread('/content/frames/'+str(i-1)+'.png')


  except:
    continue 

In [ ]:
%%time
frames = os.listdir('frames/')
frames.sort(key=lambda f: int(re.sub('\D', '', f)))

frame_array=[]

for i in range(len(frames)):
    
    #reading each files
    img = cv2.imread('frames/'+frames[i])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)

out = cv2.VideoWriter('sample_output_5sec_red.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 25, size)
 
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()